In [112]:
import numpy as np
from matplotlib import pyplot as plt 

## Iterated Best Response for finding Equilibria
In the setting of a 2 player game we wish to calculate the best response to a fixed strategy and iterate on it. Hopefully by doing so we will uncover the Nash Equilibrium for this particular game.

### Mathematics behind best response
So far as we have 4 categories that we determine they are superior in order Q1 > Q2 > Q3 > Q4. So we have to do our expectation going down in the game.

Let there be n applicants, broken into 4 categories, with d1, d2, d3, d4 size respectively.
f(mean, high) = frequency of high conventional metric talent (d1,d2) = (d1 + d2) / n
f(std, high) = frequency of high unconventional metric talent = d1/(d1+d2) = d3/(d3+d4)
f(mean, low) = 1-f(mean, high)
f(std, low) = 1-f(std, high)

Multiplying the 2 frequencies together allows us to recover the frequency of any individual category.

We define a variable x, which we call our wanted selections in total. Initially, we constrain x < 1/2 * n * f(mean, high)

We make the assumption that when a student is given an admission by both institutions, they will accept the offer from the superior institution with probability p, and 1-p in the other case.

We also make the assumption that for the institution that has p >= 1/2, they cannot distinguish between Q1 and Q2, so their offers in these respective categories break down wrt f(std, high). 

### Optimal policy for superior institution.
Since x < 1/2 * n * f(mean,high), the superior institution breaks its admissions at random into Q1 and Q2.

We keep track of the number of admissions handed out per category with a_i({OD, UD}). This also represents the strategy each player will be using in the "game" of admissions.

So, the naive and blind strategy for the OD after 0 best responses is:
a_1(OD) = x * f(std, high)
a_2(OD) = x * (1 - f(std, high))
a_3(OD) = 0
a_4(OD) = 0

### BR general form:
The underdog knows what the overdog institution would do in the naive case, so they adjust their strategy. Below we describe the algorithm to determine the best response strategy.

if x > 0 (we can still give out more admissions)
for best_category in categories:
    if x > d_(best_category) - a_(best_category):
        set a_(best_category) = d_(best_category)
        update x by calculating the expected number of students attending using a_(best)(OD) and a_(best)(UD). Subtract the increase in students
    else:
        calculate how many to accept for the final category

In this algorithm, we have to calculate how many we can accept. We can construct this backwards from our equations. First, let us see how to calculate the number of collisions/number of expected students.

### Calculating attendees
We can calculate the number of attendees y = selected(1*(1-a_i(OD)/d_i + p*(a_i(OD)/d_i))). Given y, we can calculate this value.
We also use this equation to calculate by how much we should reduce our expected number of incoming students.



In [113]:
# specifying environment variables and the functions athat allow us to run multiple experiments


frequency_dictionary = {
    "f_sigma" : 1/4,
    "f_mu" : 2/5
}
f_sigma = frequency_dictionary["f_sigma"] # the percentage of students that are unconvetionally talented
f_mu = frequency_dictionary["f_mu"] # the percentage of students that are conventionally talented

n = 100 # the number of applicants
x = 1/3 * f_mu * n # the amount of students we're seeking to admit
p = 1/3 # probability of underdog winning competition
def set_population_params(total_students: int, to_admit: float, prob_underdog: float) -> None:
    global n, x, p
    n = total_students
    x = to_admit
    p = prob_underdog

size_dictionary = {}


# specify the sizes of each section in terms of integers
def set_dict_size():
    global size_dictionary
    f_sigma = frequency_dictionary["f_sigma"] # the percentage of students that are unconvetionally talented
    f_mu = frequency_dictionary["f_mu"] # the percentage of students that are conventionally talented
    size_dictionary = {
        "Q1":f_sigma * f_mu * n,
        "Q2":(1-f_sigma) * f_mu * n,
        "Q3":f_sigma * (1-f_mu) * n,
        "Q4":(1-f_sigma) * (1-f_mu) * n
    }

# specify the frequencies of each category type and then call the function to set the actual integers used
def set_freq(mu_high=2/5, sigma_high=1/4):
    global frequency_dictionary
    frequency_dictionary = {
        "f_sigma" : sigma_high,
        "f_mu" : mu_high
    }

    set_dict_size()



strategy_dictionary = {}

# set the strategies to zero between runs
def reset_strategies():
    # specify the agent strategies
    global strategy_dictionary
    strategy_dictionary = {
    "overdog":{
        "Q1":0,
        "Q2":0,
        "Q3":0,
        "Q4":0
    }, 
    "underdog":{
        "Q1":0,
        "Q2":0,
        "Q3":0,
        "Q4":0
    }}



# specify the parameters from distribution sampling
distro_params = {}
cat_to_distro = {}
def set_distro_params(mu_high: float, mu_low: float, sigma_high: float, sigma_low: float, sigma_high_bonus: float) -> None:
    '''High level function used to set our paramaters to whatever we like of the distributions of the different categories.
    
    Useful for running experiments with different proportions betweeen the different categories to model scenarios in which different features have greater benefits.
    
    Controlled by the ratio between mu_high and mu_low as well as sigma_high and sigma_low
    
    Inputs:
        mu_high: the mean value of a distribution that has the higher mu value
        mu_low: the mean value of a distribution that has the lower mu value
        sigma_high: the standard deviation of a distribution that has higher variance
        sigma_low: the standard deviation of a distribution that has a lower variance
        sigma_high_bonus: the bonus yielded to mu values from having a higher variance candidate
        
    Output:
        None: instead updates the dictionary that contains this information'''
    global distro_params
    distro_params = {
        "mu_high" : mu_high,
        "mu_low" : mu_low,
        "sigma_high" : sigma_high,
        "sigma_low" : sigma_low
    }

    distro_params["sigma_high_bonus"] =  sigma_high_bonus * (distro_params["mu_high"]-distro_params["mu_low"])
    global cat_to_distro
    cat_to_distro = {
        "Q1" : [distro_params["mu_high"]+distro_params["sigma_high_bonus"],distro_params["sigma_high"]],
        "Q2" : [distro_params["mu_high"], distro_params["sigma_low"]],
        "Q3" : [distro_params["mu_low"]+distro_params["sigma_high_bonus"],distro_params["sigma_high"]],
        "Q4" : [distro_params["mu_low"], distro_params["sigma_low"]],
    }

set_distro_params(5,4,2,1,0.1) # just use this for a default run, then later we can govern what goes on elsewhere
set_freq()
set_population_params(100, 1/3 * frequency_dictionary["f_mu"], 1/3)


reset_strategies()

print(size_dictionary)
print(x, f_sigma)
print(strategy_dictionary)


{'Q1': 10.0, 'Q2': 30.000000000000004, 'Q3': 15.0, 'Q4': 44.99999999999999}
0.13333333333333333 0.25
{'overdog': {'Q1': 0, 'Q2': 0, 'Q3': 0, 'Q4': 0}, 'underdog': {'Q1': 0, 'Q2': 0, 'Q3': 0, 'Q4': 0}}


In [114]:
# In this cell we calculate the iterated best responses

def calc_collisions(category, strategies:dict)->float:
    numSelect = strategies["overdog"][category]
    numSelect2 = strategies["underdog"][category]
    collisions = numSelect * numSelect2 / size_dictionary[category]
    #print("num collisions", collisions)
    return collisions

# first we specify our function for calculating how much we should select
def calcSelectionNotBlind(desired_admits, strategies:dict, category_size:int, category:str, player:str, opposition:str) -> int:
    '''calcSelectionNotBlind calculates the number of students to admit from a particular category given the agent is not blind
    
    Inputs:
        desired_admits: float; how many students we want in expectation
        strategies: dict; the strategy dictionary of both players
        category_size: integer; the size of our given category
        category: string; the name of our category, used to access strategies
        player: string; the name of the player
        opposition: string; the name of the opposition
    
    Returns
        integer: returns the number of people remaining as desired admittees after iterating through the category
    '''
    
    # if we have 0 desired admits then just return 0, we have this due to the fact that numbers are automatically passed down
    if desired_admits == 0 or desired_admits == float(0):
        return 0
    
    # Renaming trick with q and p to be able to accommodate both over and underdog in one formula later on
    if player == "overdog":
        q = 1-p
    else:
        q = p

    # We examine how likely our opposition is to select from this category, we store this as a probability
    opposition_selection_prob = strategies[opposition][category]/size_dictionary[category]
    intial_selection_number = strategies[player][category]

    # If we know we want to admit more people than exist in this category, just set our strategy to admit all of them.
    # this is only doable because we're going down the categories.
    if desired_admits > category_size:
        strategies[player][category] = category_size
    else:
        # If there are less desired admits than people in the category, we use our formula to determine how many we actually should admit.
        to_add = desired_admits/(1-opposition_selection_prob+q*opposition_selection_prob)
        #print("want to admit", to_add)

        # We again check if this value is greater than the category size, if yes, then just maximize it
        if to_add > category_size: # in the case we would go over (possible if selection frequency is high) cap it
            strategies[player][category] = category_size
        else:
            strategies[player][category] = to_add

    # in this step we calculate the difference between our starting desired admits and how many people we are expecting to ultimately admit in this category.
    return desired_admits - (strategies[player][category] - calc_collisions(category, strategies))




def find_best_responseNotBlind(desired_admits, strategies:dict, player:str, opposition:str):
    # in this function we loop through all categories in order to find the best response

    for cat in size_dictionary.keys():
        #print(size_dictionary[cat])
        desired_admits = calcSelectionNotBlind(desired_admits, strategies, size_dictionary[cat], cat, player, opposition)
        #print()
        #print("desired_admits selections remaining", desired_admits)


print("before",strategy_dictionary)
#find_best_responseNotBlind(x, strategy_dictionary, "underdog", "overdog")
print("after",strategy_dictionary)


before {'overdog': {'Q1': 0, 'Q2': 0, 'Q3': 0, 'Q4': 0}, 'underdog': {'Q1': 0, 'Q2': 0, 'Q3': 0, 'Q4': 0}}
after {'overdog': {'Q1': 0, 'Q2': 0, 'Q3': 0, 'Q4': 0}, 'underdog': {'Q1': 0, 'Q2': 0, 'Q3': 0, 'Q4': 0}}


In [115]:
def calcSelectionBlind(desired_admits, strategies:dict, player:str, opposition:str, mu="high"):
    if desired_admits == 0 or desired_admits == float(0):
        return 0
    
    # 
    if mu == "high":
        # merge the upper two categories to get our needed category
        category_size = size_dictionary["Q1"] + size_dictionary["Q2"]
        opposition_selection_prob = (strategies[opposition]["Q1"] + strategies[opposition]["Q2"])/(category_size)
    else:
        category_size = size_dictionary["Q3"] + size_dictionary["Q4"]
        opposition_selection_prob = (strategies[opposition]["Q3"] + strategies[opposition]["Q4"])/(category_size)

    # if we wish to admit more people than there are in the category, then 
    if desired_admits > category_size:
        breakdownBlindSelection(category_size, mu, strategies, player)


    else:
        # in the case its less, use the formula specified
        to_add = desired_admits/(1-opposition_selection_prob+(1-p)*opposition_selection_prob)
        #print("want to admit", to_add)
        if to_add > category_size: # in the case we would go over (possible if selection frequency is high) cap it
            breakdownBlindSelection(category_size, mu, strategies, player)
        else:
            breakdownBlindSelection(to_add, mu, strategies, player)

    if mu == "high":
        expected1 = strategies[player]["Q1"] - calc_collisions("Q1", strategies)
        expected2 = strategies[player]["Q2"] - calc_collisions("Q2", strategies)
    else:
        expected1 = strategies[player]["Q3"] - calc_collisions("Q3", strategies)
        expected2 = strategies[player]["Q4"] - calc_collisions("Q4", strategies)
    return desired_admits - expected1 - expected2

def breakdownBlindSelection(number, mu, strategies, player):
    num1 = number * f_sigma
    num2 = number * (1-f_sigma)
    if mu == "high":
        strategies[player]["Q1"] = num1
        strategies[player]["Q2"] = num2
    else:
        strategies[player]["Q3"] = num1
        strategies[player]["Q4"] = num2

def find_best_responseBlind(desired_admits, strategies:dict, player:str, opposition:str):
    # in this function we loop through all categories in order to find the best response
    mus = ["high", "low"]
    for mu in mus:
        desired_admits = calcSelectionBlind(desired_admits, strategies, player, opposition, mu)
        #print()
        #print("desired_admits selections remaining", desired_admits)


## Find equilibria
Keep iterating until strategies don't change

In [116]:


def copy_strategy(strategies, player):
    return list(strategies[player].values())

def find_strategies(p_val:float, blind=False, naive=100, reverse=False) -> None:
    '''
    This functions task is to find and return the strategieis that will be employed by the players performing iterative best response to one another's moves.
    
    Inputs:
        p_val: float; represents the breakdown probability of a collision where p is the chance of the underdog winning
        blind: boolean; represents whether our overdog player is blind or not
        naive: integer; represents how many iterations of best response they should go through. A naive value of 1 allows 1 best response
        reverse: boolean; represents who gets to go first in terms of best response iterations. If False then the underdog goes first.

        
    Returns: None; Instead updates the strategy dictionaries for both players
    '''
    # reset the strategies to 0
    reset_strategies()
    change = 100000
    global p
    p = p_val
    iterations = 0
    while change > 0.001 or iterations < naive:
        strat_vals = copy_strategy(strategy_dictionary, "overdog")

        #print(strat_vals) # print strategy values
        if not reverse:
            find_best_responseNotBlind(x, strategy_dictionary, "underdog", "overdog")
            if blind:
                find_best_responseBlind(x, strategy_dictionary, "overdog", "underdog")
            else:
                find_best_responseNotBlind(x, strategy_dictionary, "overdog", "underdog")

        else:
            if blind:
                find_best_responseBlind(x, strategy_dictionary, "overdog", "underdog")
            else:
                find_best_responseNotBlind(x, strategy_dictionary, "overdog", "underdog")
            find_best_responseNotBlind(x, strategy_dictionary, "underdog", "overdog")

        updated = copy_strategy(strategy_dictionary, "overdog")


        ''' Debugging information
        print(strat_vals)
        print(updated)
        totalCollisions = sum([calc_collisions(key, strategy_dictionary) for key in cat_to_distro.keys()])
        print("Expected number of collisions is", totalCollisions)
        print("Total attempted admissions is", sum(updated))
        print("Admissions - collisions",  sum(updated) - totalCollisions)
        print(x)
        if iterations == 0:
            #print(strategy_dictionary["overdog"])

        '''

        result = [a_i - b_i for a_i, b_i in zip(updated, strat_vals)]
        change = max(result)
        iterations += 1
    #print()
    #print("converged to equilibrium in", iterations, "iterations")
    #print(strategy_dictionary)


find_strategies(p, blind=False)
find_strategies(p, blind=False, reverse=True)
#print()
#print("converged to equilibrium in", iterations, "iterations")
#print(strategy_dictionary)



## Play the game
We implement the actual game result, to see how much advantage can be gained

In [117]:
def get_proportion(result):
    ''' returns the proportion of the underdog achieved'''
    return result["under"]/(result["over"]+result["under"])
def play_game(strategies:dict) -> dict:
    '''This function takes the strategies of both players in the form of a dictionary and simulates a game playing out. Outputting a dictionary with the same player names and the returned achieved value.
    Input:
        strategies : dict
    Output:
        results : dict'''
    
    # for each player calculate the number of rolls they have in each category and simulate them - round to the nearest whole number. We then take the average of the result

    over_strats = strategies["overdog"]
    under_strats = strategies["underdog"]

   # print(over_strats)
   # print(under_strats)
    
    overStudents = []
    underStudents = []
    for key in over_strats.keys():
        #print("category size", size_dictionary[key])
        wantedRollsOver = over_strats[key]
        #print(wantedRollsOver)
        wantedRollsUnder = under_strats[key]
        collisions = calc_collisions(key, strategies)
        overRolls = wantedRollsOver - collisions*(p) # have to invert these since we're subtracting the number of cases in which we don't win
        underRolls = wantedRollsUnder - collisions*(1-p)

        # get the mu and sigma of both elements
        mu, sigma = cat_to_distro[key][0], cat_to_distro[key][1]
        
        # simulate real student talents and add them to the list of students
        #print("iterated length overstudents", len(overStudents))
        overStudents = np.concatenate((overStudents, np.random.normal(loc=mu, scale=sigma, size=max(0, round(overRolls)))))
        underStudents = np.concatenate((underStudents,np.random.normal(loc=mu, scale=sigma, size=max(0,round(underRolls)))))
    
    #print("final len overstudents", len(overStudents))
    #print(underStudents)
    result = {
        "over" : np.mean(overStudents),
        "under" : np.mean(underStudents)
    }

    return result


# simulate 100 games
proportions = []
def get_game_results(num_students: int, f_mu: float, f_sigma: float, to_admit: float, blind: bool, mu_high_val: float, mu_low_val: float, sigma_high_val: float, sigma_low_val: float, sigma_high_bonus_val: float, prob_underdog: float, expected_val: bool) -> float:
    '''Simulate a game with particular environment setup 30 times to get a decent idea of the average result'''
    # set up the game environment
    set_population_params(num_students, to_admit, prob_underdog)
    set_distro_params(mu_high=mu_high_val,mu_low=mu_low_val,sigma_high=sigma_high_val,sigma_low=sigma_low_val,sigma_high_bonus=sigma_high_bonus_val) # pass these values
    set_freq(mu_high=f_mu, sigma_high=f_sigma) # set the frequencies
    
    proportions = []
    for i in range(100):
        find_strategies(p, blind=blind)
        #print(strategy_dictionary)
        result = play_game(strategy_dictionary)
        #print(result)
        #print(get_proportion(result))
        proportions.append(get_proportion(result))
    #print(result)
    #print(proportions)
    return np.mean(proportions)

def get_game_results_multi(tup) -> float:
    raise ValueError('Not yet implemented, need to modify the function \'find_strategies\' and \'play_game\' first to work with locally passed variables instead of global variables!')


In [118]:
value = get_game_results(num_students=100, f_mu=1/10, f_sigma=1/4, to_admit=1/10*1/10*100, blind=True, mu_high_val=6, mu_low_val=4, sigma_high_val=2, sigma_low_val=1, sigma_high_bonus_val=0.5, prob_underdog=1/5, expected_val=True)
print(value)

0.5340821666228919


### Running experiments and exporting them to CSV
Going to alter the probability win in collisions as one variable to keep fixed. Then the other variables we will test will be ratio of high_mu relative to number of admits, blind or not blind, and f_sigma high proportion

In [119]:
# Setup the experiment to make the different runs, store the environment variables and the result as a line in a list
mu_high_props = [i/100 for i in range(5, 100, 3)]
sigma_high_props = mu_high_props[:]

to_admit_prop = mu_high_props[:] # multiply this with the size of students and the frequency of mu_high to get the number of students to admit
# if the value is stable in mu_high props in terms of proportion gained back we know we can ignore it

sigma_bonuses = mu_high_props[:]

p_underdogs = mu_high_props[:]

mu_higher_value_ratio = mu_high_props[:]

blinds = [False, True]

# some default values to make testing faster
mu_high = 1/3
sigma_high = 1/4
mu_default = 10
#


toExport = [["blind", "p_underdog", "proportion_mu_high_to_admit", "mu_high", "sigma_high", "sigma_bonus", "mu_high_pct_higher", "game_value"]]
index = 1
total = 2 * (len(p_underdogs)**6)
print(len(mu_high_props), total)

# blind settings
for blind in blinds:

    # probability that the underdog wins
    for p in p_underdogs:

        # proportion of students to admit
        for to_admit in to_admit_prop:

            # proportion of students that are high in talent by UNCONVENTIONAL metrics
            for sigma_high in sigma_high_props:

                # proportion of students that are high in talent by CONVENTIONAL metrics
                for mu_high in mu_high_props:

                    # the proportion of the difference between the average value conventionally high talent students and low talent students that is made up for by unconventional talent
                    # will work directly with blindness. Without blindness there is no difference
                    for sigma_bonus in sigma_bonuses:

                        #
                        for mu_higher in mu_higher_value_ratio:

                            # the number of students we want to admit
                            to_admit_val = to_admit * mu_high * n

                            # plays a game with the particular settings and returns the outcome
                            value_game = get_game_results(100, mu_high, sigma_high, to_admit_val, blind, mu_default*(1+mu_higher), mu_default, 2, 1, sigma_bonus, p, True)

                            # add a row to our output
                            toExport.append([blind, p, to_admit, mu_high, sigma_high, sigma_bonus, mu_higher, value_game])
                            #print([blind, p, to_admit, mu_high, sigma_high, sigma_bonus, value_game])

                            if index % 50 == 0:
                                print("Percentage done:", index/total*100)
                            index += 1

import csv

def export_to_csv(data, filename):
    with open(filename, 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        
        # Writing column names
        csv_writer.writerow(data[0])
        
        # Writing data rows
        for row in data[1:]:
            csv_writer.writerow(row)

export_to_csv(toExport, "results/output2.csv")




19 4952198


c:\Users\csong\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\csong\anaconda3\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Percentage done: 0.0010096526835154814
Percentage done: 0.002019305367030963
Percentage done: 0.003028958050546444
Percentage done: 0.004038610734061926
Percentage done: 0.005048263417577407
Percentage done: 0.006057916101092888
Percentage done: 0.00706756878460837
Percentage done: 0.008077221468123851
Percentage done: 0.009086874151639333
Percentage done: 0.010096526835154815
Percentage done: 0.011106179518670295
Percentage done: 0.012115832202185777
Percentage done: 0.013125484885701259
Percentage done: 0.01413513756921674
Percentage done: 0.015144790252732222
Percentage done: 0.016154442936247702
Percentage done: 0.017164095619763186
Percentage done: 0.018173748303278666
Percentage done: 0.019183400986794146
Percentage done: 0.02019305367030963
Percentage done: 0.02120270635382511
Percentage done: 0.02221235903734059
Percentage done: 0.023222011720856073
Percentage done: 0.024231664404371554
Percentage done: 0.025241317087887034
Percentage done: 0.026250969771402517
Percentage done:

KeyboardInterrupt: 